In [454]:
import pandas as pd
import numpy as np

In [455]:
df = pd.read_csv(r'shedulers.csv', encoding='cp1251', sep=';')

In [456]:
df

,ФИО,Расписание,Дата начала расписания,Дата окончания расписания
0,Поставщик 1,дддвсвнн,01.01.2019 0:00,10.01.2019 0:00
1,Поставщик 1,ннвннв,11.01.2019 0:00,15.01.2019 0:00
2,Поставщик 1,св,16.01.2019 0:00,20.01.2019 0:00
3,Поставщик 2,свсвсв,01.01.2019 0:00,07.01.2019 0:00
4,Поставщик 2,днвсв,08.01.2019 0:00,14.01.2019 0:00
5,Поставщик 2,ннддвсв,15.01.2019 0:00,31.12.9999 0:00
6,Поставщик 3,нвнвнв,01.01.2019 0:00,01.02.2019 0:00
7,Поставщик 3,двдвдвдв,02.02.2019 0:00,31.12.9999 0:00


In [457]:
df.shape

(8, 4)

In [458]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8 entries, 0 to 7
Data columns (total 4 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   ФИО                        8 non-null      object
 1   Расписание                 8 non-null      object
 2   Дата начала расписания     8 non-null      object
 3   Дата окончания расписания  8 non-null      object
dtypes: object(4)
memory usage: 384.0+ bytes


In [459]:
df.iloc[5, 3] = '31.12.2019 0:00'
df.iloc[7, 3] = '31.12.2019 0:00'

## Задание 1
Создать таблицу (T_CONTRACTOR_SHERULER) под расписание и заполнить его с файла schedulers.csv. (Использовать любую библиотеку для заливки в СУБД (как вариант MS SQL)) Таблицу следует нормализовать.
ID_NAME - идентификтор поставщика
NAME - название поставщика
SHEDULE - расписание
DATE_BEGIN - дата начала действия расписания
DATE_END - дата окончания действия расписания
Пример записи без нормализации:
Поставщик 1 ДВС 01.01.2019 04.01.2019
Поставщик 2 НВС 05.01.2019 31.12.2019
Вводные:
Связку полей FIO, DATE_BEGIN считать уникальной.
DATE_BEGIN не может привышать DATE_END.
Можете продемонстрировать работу с ключами/ограничениями.

In [278]:
import csv
import sqlite3

In [279]:
con = sqlite3.connect(":memory:")
cur = con.cursor()

In [280]:
cur.execute("CREATE TABLE T_CONTRACTOR_SHERULER (ID_NAME INT, NAME STR, SHEDULE STR, \
            DATE_BEGIN STR CHECK(DATE_BEGIN < DATE_END) , DATE_END STR, \
            PRIMARY KEY(NAME, DATE_BEGIN))") 

In [281]:
import regex as re
to_db = [((int(re.search(r'\d+', df.iloc[i, :]['ФИО']).group(0))), df.iloc[i, :]['ФИО'], \
          df.iloc[i, :]['Расписание'], df.iloc[i, :]['Дата начала расписания'], \
          df.iloc[i, :]['Дата окончания расписания']) for i in range(df.shape[0])]
cur.executemany("INSERT INTO T_CONTRACTOR_SHERULER VALUES (?,?,?,?,?);", to_db)
con.commit()

In [282]:
with con:   
    cur.execute("SELECT * FROM T_CONTRACTOR_SHERULER")
    rows = cur.fetchall()
 
    for row in rows:
        print (row)

(1, 'Поставщик 1', 'дддвсвнн', '01.01.2019 0:00', '10.01.2019 0:00')
(1, 'Поставщик 1', 'ннвннв', '11.01.2019 0:00', '15.01.2019 0:00')
(1, 'Поставщик 1', 'св', '16.01.2019 0:00', '20.01.2019 0:00')
(2, 'Поставщик 2', 'свсвсв', '01.01.2019 0:00', '07.01.2019 0:00')
(2, 'Поставщик 2', 'днвсв', '08.01.2019 0:00', '14.01.2019 0:00')
(2, 'Поставщик 2', 'ннддвсв', '15.01.2019 0:00', '31.12.2019 0:00')
(3, 'Поставщик 3', 'нвнвнв', '01.01.2019 0:00', '01.02.2019 0:00')
(3, 'Поставщик 3', 'двдвдвдв', '02.02.2019 0:00', '31.12.2019 0:00')


## Задание 2
Создать таблицу (T_CONTRACTOR_WORK_DAY) выходов на работу сотрудников.
Таблица должна иметь следующий вид
ID - идентификатор записи
NAME - название поставщика
DATE_BEGIN - Начало рабочего дня (datetime)
DATE_END - Конец рабочего дня (datetime)

In [283]:
cur.execute("CREATE TABLE T_CONTRACTOR_WORK_DAY (\
            ID INT, NAME STR, DATE_BEGIN STR CHECK(DATE_BEGIN < DATE_END) , DATE_END STR)") 

## Задание 3
Создать процедуру расчета рабочих дней.
Входящие параметры:
Дата начала периода расчета
Дата окончания периода расчета.
Ожидаемый результать выполнения хранимой процедуры - заполнение таблицы T_CONTRACTOR_WORK_DAY рабочими днями согласно расписания работы поставщиков из таблицы T_CONTRACTOR_SHERULER Выходные дни (В) не должны попадать в таблицу T_CONTRACTOR_WORK_DAY

In [284]:
cur.execute("CREATE TABLE SHEDULERS (TYPE STR, BEGIN STR, END STR)") 

In [285]:
to_db = [('д', '08:00', '20:00'),
        ('н', '20:00', '08:00'),
        ('с', '08:00', '08:00')]

In [286]:
cur.executemany("INSERT INTO SHEDULERS VALUES (?,?,?);", to_db)
con.commit()

In [287]:
with con:   
    cur.execute("SELECT * FROM SHEDULERS")
    rows = cur.fetchall()
 
    for row in rows:
        print (row)

('д', '08:00', '20:00')
('н', '20:00', '08:00')
('с', '08:00', '08:00')


## Задание 5

Выполните задачи 3, 4 с помощью python.

In [460]:
df['Дата начала расписания'] = pd.to_datetime(df['Дата начала расписания'], format="%d.%m.%Y %H:%M")
df['Дата окончания расписания'] = pd.to_datetime(df['Дата окончания расписания'], format="%d.%m.%Y %H:%M")

In [461]:
df

,ФИО,Расписание,Дата начала расписания,Дата окончания расписания
0,Поставщик 1,дддвсвнн,2019-01-01,2019-01-10
1,Поставщик 1,ннвннв,2019-01-11,2019-01-15
2,Поставщик 1,св,2019-01-16,2019-01-20
3,Поставщик 2,свсвсв,2019-01-01,2019-01-07
4,Поставщик 2,днвсв,2019-01-08,2019-01-14
5,Поставщик 2,ннддвсв,2019-01-15,2019-12-31
6,Поставщик 3,нвнвнв,2019-01-01,2019-02-01
7,Поставщик 3,двдвдвдв,2019-02-02,2019-12-31


In [516]:
df_new = pd.DataFrame()

for i in range(df.shape[0]):
    df_temp = pd.DataFrame()
    lenght = abs(df.iloc[i,3]-df.iloc[i,2]).days+1
    list_of_shed = list(df.iloc[i,1])
    temp_len = len(list_of_shed)
    name_list = []
    type_shed = []
    start_list = []
    end_list = []
    for j in range(lenght):
        name_list.append(df.iloc[i, 0])
        start_list.append(df.iloc[i, 2]+datetime.timedelta(days=j))
        #end_list.append(df.iloc[i, 2]+datetime.timedelta(days=j+1))
        try:
            type_shed.append(list_of_shed[j])
        except:
            while j >= temp_len:
                j = j - temp_len
            type_shed.append(list_of_shed[j])
    df_temp['name'] = name_list
    df_temp['type_shed'] = type_shed
    df_temp['start'] = start_list
    #df_temp['end'] = end_list
    #if df_temp.iloc[-1, 1] in ['с', 'н']:
     #   df_temp.drop(df_temp.tail(1).index,inplace=True)
    df_new = df_new.append(df_temp)
     
        
def startWorkTimeDefinder(row):
    if row[1] != 'н':
        time_start = row[2] + datetime.timedelta(hours=8)
    else:
        time_start = row[2] + datetime.timedelta(hours=20)    
    return time_start


def endWorkTimeDefinder(row):
    if (row[1] != 'с'):
        time_end = row[2] + datetime.timedelta(hours=12)
    else:
        time_end = row[2] + datetime.timedelta(hours=24)    
    return time_end

    
df_new = df_new[df_new.type_shed != 'в'].reset_index(drop=True) 
df_new['start'] = df_new.apply(startWorkTimeDefinder, axis=1)
df_new['end'] = df_new.apply(endWorkTimeDefinder, axis=1)


In [517]:
df_new = df_new.drop('type_shed', axis=1)

In [519]:
df_new.head(20)

,name,start,end
0,Поставщик 1,2019-01-01 08:00:00,2019-01-01 20:00:00
1,Поставщик 1,2019-01-02 08:00:00,2019-01-02 20:00:00
2,Поставщик 1,2019-01-03 08:00:00,2019-01-03 20:00:00
3,Поставщик 1,2019-01-05 08:00:00,2019-01-06 08:00:00
4,Поставщик 1,2019-01-07 20:00:00,2019-01-08 08:00:00
5,Поставщик 1,2019-01-08 20:00:00,2019-01-09 08:00:00
6,Поставщик 1,2019-01-09 08:00:00,2019-01-09 20:00:00
7,Поставщик 1,2019-01-10 08:00:00,2019-01-10 20:00:00
8,Поставщик 1,2019-01-11 20:00:00,2019-01-12 08:00:00
9,Поставщик 1,2019-01-12 20:00:00,2019-01-13 08:00:00


Cделать выборку содержащую сколько рабочих дней было у каждого поставщика

In [520]:
df_new.groupby('name', as_index=False).agg({'start':'count'}).rename(columns={'start':'count_work_days'})

,name,count_work_days
0,Поставщик 1,15
1,Поставщик 2,260
2,Поставщик 3,183


Сделать выборку поставщиков, у которых было больше 10 рабочих дней за январь 2019 года

In [521]:
df_new.query("start.dt.month == 1 & start.dt.year == 2019")\
            .groupby('name', as_index=False).agg({'start':'count'}).query("start>10").name

0    Поставщик 1
1    Поставщик 2
2    Поставщик 3
Name: name, dtype: object

Сделать выборку поставщиков, кто работал 14, 15 и 16 января 2019 года

In [522]:
list_of_dates = [datetime.date(2019, 1, 14), datetime.date(2019, 1, 15), 
                 datetime.date(2019, 1, 16)]
df_new.query("start.dt.date in @list_of_dates | end.dt.date in @list_of_dates")\
                .name.unique()

array(['Поставщик 1', 'Поставщик 2', 'Поставщик 3'], dtype=object)